In [158]:
class Note:
    
    def __init__(self, channel, pitch, velocity, timestamp, duration):
        self.channel = channel
        self.pitch = pitch
        self.velocity = velocity
        
        # in MIDI 'ticks'
        self.timestamp = timestamp
        self.duration = duration
        
    def __eq__(self, other):
        return self.channel == other.channel and self.pitch == other.pitch and self.velocity == other.velocity and \
            self.duration == other.duration
        
    def __repr__(self):
        return f"[Note {self.channel} {self.pitch} {self.velocity} {self.duration}]"
        
    def __hash__(self):
        return hash(str(self))

In [159]:
from random import randint
from midiutil import MIDIFile

class MarkovGenerator:
    
    def __init__(self, order, note_lst):
        self.order = order
        self.note_lst = note_lst
        self.adj = {}
        
        self.build_graph()
        
    def build_graph(self):
        
        for i in range(len(note_lst)):
            
            for j in range(i+1, i+1+self.order):
                
                note_slice = tuple(note_lst[i:j])
                
                if j+1 >= len(note_lst):
                    # out of bounds
                    break
                    
                nxt_note = note_lst[j+1]
                
                if note_slice not in self.adj:
                    self.adj[note_slice] = []
                    
                self.adj[note_slice].append(nxt_note)
                
    def generate_notes(self, song_length):
        
        output_notes = [self.note_lst[randint(0, len(self.note_lst) - 1)]]
        
        for _ in range(song_length):
            
            start_idx = max(0, len(output_notes) - self.order)
            end_idx = min(len(output_notes), start_idx + self.order)
            
            while start_idx < end_idx:
                
                note_slice = tuple(output_notes[start_idx: end_idx])
                
                # print("trying", note_slice)
            
                if note_slice in self.adj:
                    
                    # found a match in our adjacency list
                    
                    # print("using", start_idx, end_idx, note_slice, output_notes[-3:])
                    
                    options = self.adj[note_slice]
                    output_notes.append(options[randint(0, len(options) - 1)])
                    
                    break
                
                start_idx += 1
        
        return output_notes
    
    def generate_midi(self, song_length):
    
        notes = self.generate_notes(song_length)

        track = 0
        tempo = 60 # In BPM

        MyMIDI = MIDIFile(1, eventtime_is_ticks=True)
        MyMIDI.addTempo(track,time,tempo)

        for note in notes:

            MyMIDI.addNote(track, note.channel, note.pitch, note.timestamp, note.duration, note.velocity)

        return MyMIDI

In [160]:
from mido import Message, MidiFile, MidiTrack

mid = MidiFile('the_real_folk_blues.mid')

note_set = set()
note_lst = []

open_notes = {}

counter = 0
cur_tick = 0

for msg in mid.tracks[0]:
    
    cur_tick += msg.time
    
    if msg.type == 'note_on' and msg.velocity != 0:
       
        note = Note(msg.channel, msg.note, msg.velocity, cur_tick, 0)
        open_notes[(msg.channel, msg.note)] = note
        counter += 1
        
    elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
        
        note_id = (msg.channel, msg.note)
        
        if note_id in open_notes:
            open_notes[note_id].duration = cur_tick - open_notes[note_id].timestamp
            note_set.add(open_notes[note_id])
            note_lst.append(open_notes[note_id])
            
            open_notes.pop(note_id)
        
print(f"Total notes: {counter}")
print(f"Unique notes: {len(note_set)}")

note_lst.sort(key=lambda x: x.timestamp)

generator = MarkovGenerator(3, note_lst)
output_mid = generator.generate_midi(2000)

# output_mid = generate_midi(note_lst)

with open("folk_blue_generated.mid", "wb") as output_file:
    output_mid.writeFile(output_file)
    
print('finished')

Total notes: 2100
Unique notes: 322


NameError: name 'time' is not defined

In [156]:
from midiutil import MIDIFile

def generate_midi(notes):
    
    if not notes:
        return

    track = 0
    time = 0
    tempo = 60 # In BPM
    
    MyMIDI = MIDIFile(1, eventtime_is_ticks=True)
    MyMIDI.addTempo(track,time,tempo)
    
    for note in notes:
        
        MyMIDI.addNote(track, note.channel, note.pitch, note.timestamp, note.duration, note.velocity)
   
    return MyMIDI